## _*QISKit ACQUA Chemistry advanced how to 2*_

This notebook demonstrates how to use QISKit ACQUA Chemistry to compute the ground state energy of a Lithium Hydride (LiH) molecule using VQE and UCCSD.

This notebook shows another example about how to experiment with the ground state energy of a molecule step by step by using low-level programming API as compared to simpliy feeding a configuraiton dict as in [acqua_chemistry_howto](acqua_chemistry_howto.ipynb) example.

Here we demonstrate: **item 2** is the major difference from the notebook [acqua_chemistry_howto_advanced_1](acqua_chemistry_howto_advanced_1.ipynb)
  1. how to define a molecule and get integrals from a driver (PySCF)
  2. how to construct a qubit operator by using hamiltonian class
  3. how to get dynamically-loadded instance, such as algorithm, optimizer, variational_form, and initial_state
  4. how to configure above instance programmatically
  5. how to retrieve the results from the algorithm

In [1]:
# import common packages
import numpy as np
from collections import OrderedDict

# lib from QISKit ACQUA Chemistry
from qiskit_acqua_chemistry.core import get_chemistry_operator_instance

# lib for driver
from qiskit_acqua_chemistry.drivers import ConfigurationManager

# lib from QISKit ACQUA
from qiskit_acqua.operator import Operator
from qiskit_acqua import (get_algorithm_instance, get_optimizer_instance, get_variational_form_instance, get_initial_state_instance)

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'Li .0 .0 .0; H .0 .0 1.6'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [6]:
core = get_chemistry_operator_instance('hamiltonian')
hamiltonian_cfg = OrderedDict([
    ('name', 'hamiltonian'),
    ('transformation', 'full'),
    ('qubit_mapping', 'parity'),
    ('two_qubit_reduction', True),
    ('freeze_core', True),
    ('orbital_reduction', [-3, -2])
])
core.init_params(hamiltonian_cfg)
algo_input = core.run(molecule)
qubitOp = algo_input.qubit_op

energy_shift = core._energy_shift
nuclear_repulsion_energy = core._nuclear_repulsion_energy

num_spin_orbitals = core._molecule_info['num_orbitals']
num_particles = core._molecule_info['num_particles']
qubit_reduction = core._two_qubit_reduction
map_type = core._qubit_mapping

In [7]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The computed energy is: {:.12f}'.format(ret['eigvals'][0].real))
print('The exact ground state energy is: {:.12f}'.format(ret['eigvals'][0].real + energy_shift + nuclear_repulsion_energy))

The computed energy is: -1.077059745735
The exact ground state energy is: -7.881072044031


In [9]:
# setup VQE 
# setup optimizer, use COBYLA optimizer for example
max_eval = 200
cobyla = get_optimizer_instance('COBYLA')
cobyla.set_options(maxiter=max_eval)

# setup variation form generator (generate trial circuits for VQE)
HF_state = get_initial_state_instance('HartreeFock')
HF_state.init_args(qubitOp.num_qubits, num_spin_orbitals, map_type, qubit_reduction, num_particles)
var_form = get_variational_form_instance('UCCSD')
var_form.init_args(qubitOp.num_qubits, depth=1, num_orbitals=num_spin_orbitals, num_particles = num_particles, 
                   active_occupied=[0], active_unoccupied=[0, 1],
                   initial_state=HF_state, qubit_mapping=map_type, 
                   two_qubit_reduction=qubit_reduction, num_time_slices=1)

init_points = var_form.preferred_init_points
vqe_algorithm = get_algorithm_instance('VQE')
vqe_algorithm.setup_quantum_backend(backend='local_statevector_simulator')
vqe_algorithm.init_args(qubitOp, 'matrix', var_form, cobyla, opt_init_point=init_points)
results = vqe_algorithm.run()
print('The computed ground state energy is: {:.12f}'.format(results['eigvals'][0]))
print('The exact ground state energy is: {:.12f}'.format(results['eigvals'][0] + energy_shift + nuclear_repulsion_energy))
print("Parameters: {}".format(results['opt_params']))

The computed ground state energy is: -1.077059737908
The exact ground state energy is: -7.881072036205
Parameters: [ 0.03425824  0.0052999   0.03442233  0.00533808 -0.03878529  0.06029099
  0.06050635 -0.11631612]
